In [1]:
from datasets import DatasetDict, Dataset

DatasetDict({
    train: Dataset({
        features: ['start', 'target', 'feat_static_cat', 'feat_dynamic_real', 'item_id'],
        num_rows: 17
    })
    validation: Dataset({
        features: ['start', 'target', 'feat_static_cat', 'feat_dynamic_real', 'item_id'],
        num_rows: 17
    })
    test: Dataset({
        features: ['start', 'target', 'feat_static_cat', 'feat_dynamic_real', 'item_id'],
        num_rows: 17
    })
})

dict_keys(['start', 'target', 'feat_static_cat', 'feat_dynamic_real', 'item_id'])

2004-07-26 00:00:00
5080
